# Neo4j Notebook
Shahzeb Imtiaz
### Dataset
World Nuclear Power Reactors (Updated 7 months ago)
https://www.kaggle.com/datasets/tariqbashir/world-nuclear-power-reactors
### Collaborators
Tariq Mahmood(Owner)

### Introduction
This dataset provides a comprehensive stats about the world nuclear power generation resources. It provides detailed information about the operable nuclear power stations working globally. It comprises of location, country, reactor name, type, commissioning date, status, production of electricity per unit.

### Installation

In [ ]:
!pip3 install neo4j

### Import libraries

In [2]:
import pandas as pd
from neo4j import GraphDatabase

### Connecting to the database

In [36]:
from neo4j import GraphDatabase

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = "neo4j+s://<your url here>"
AUTH = ("username", "password")

driver = GraphDatabase.driver(URI, auth=AUTH)
driver.verify_connectivity()

### Data cleaning

In [38]:
import pandas as pd

# Define the path to the CSV file
csv_file_path = "../data/World_Nuclear_Power_Reactors.csv"

# Load the CSV file with different encodings to handle UnicodeDecodeError
encodings = ['utf-8', 'latin-1', 'ISO-8859-1']
for encoding in encodings:
    try:
        df = pd.read_csv(csv_file_path, encoding=encoding)
        break
    except UnicodeDecodeError:
        continue

# Fill missing values with the mean for numeric columns
numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
df[numeric_columns] = df[numeric_columns].fillna(df[numeric_columns].mean())

# Save the cleaned CSV file
cleaned_csv_file_path = "../data/cleaned_World_Nuclear_Power_Reactors.csv"
df.to_csv(cleaned_csv_file_path, index=False)

print("Data cleaning completed. Cleaned CSV file saved successfully.")


Data cleaning completed. Cleaned CSV file saved successfully.


Data loading

In [ ]:
import csv

# Neo4j Aura database credentials
URI = "neo4j+s://your url here"
username = "username"
password = "password"

# Function to connect to the Neo4j Aura database
def connect_to_database():
    return GraphDatabase.driver(URI, auth=(username, password))

# Function to load data from CSV into the database
def load_data(driver): 
    with open('../data/cleaned_World_Nuclear_Power_Reactors.csv', mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        with driver.session() as session:
            for row in csv_reader:
                query = """
                MERGE (country:Country { name: $countryName })
                CREATE (r:Reactor {
                    Location: $location,
                    ReactorName: $reactorName,
                    Model: $model,
                    ReactorType: $reactorType,
                    NetCapacityMWe: toFloat($netCapacityMWe),
                    ConstructionStart: $constructionStart,
                    FirstGridConnection: $firstGridConnection,
                    Status: $status
                })-[:LOCATED_IN]->(country)
                """
                session.run(query, 
                            countryName=row['Country'],
                            location=row['Location'],
                            reactorName=row['Reactor Name'],
                            model=row['Model'],
                            reactorType=row['Reactor Type'],
                            netCapacityMWe=row['Net Capacity (MWe)'],
                            constructionStart=row['Construction Start'],
                            firstGridConnection=row['First Grid Connection'],
                            status=row['Status'])

# Connect to the Neo4j Aura database
driver = connect_to_database()

# Load data from CSV into the database
load_data(driver)

### Questions and Answers

### 1. Easy: How many countries have nuclear power reactors?

In [53]:
query = """
MATCH (c:Country)<-[:LOCATED_IN]-(r:Reactor)
RETURN COUNT(DISTINCT c) AS NumberOfCountriesWithReactors
"""

result = driver.execute_query(query)
count_countries_with_reactors = result[0][0]["NumberOfCountriesWithReactors"]

print("Number of countries with nuclear power reactors:", count_countries_with_reactors)


Number of countries with nuclear power reactors: 39


### 2. Count the total number of reactors which the Status is 'Working'.

In [50]:
query = """
MATCH (r:Reactor)
WHERE r.Status = 'Working'
RETURN count(r) AS numberOfWorkingReactors
"""

result = driver.execute_query(query)
count_working_reactors = result[0][0]["numberOfWorkingReactors"]

print("Number of working reactors:", count_working_reactors)


Number of working reactors: 311


### 3. Delete the nodes which with 'Permanent Shutdown' Status and any relationship attached them.

In [54]:
query_count_before = """
MATCH (r:Reactor)
WHERE r.Status = 'Permanent Shutdown'
RETURN count(r) AS countBeforeDeletion
"""

result_before = driver.execute_query(query_count_before)
count_before = result_before[0][0]["countBeforeDeletion"]

query_delete = """
MATCH (r:Reactor)
WHERE r.Status = 'Permanent Shutdown'
DETACH DELETE r
"""

result_delete = driver.execute_query(query_delete)

query_count_after = """
MATCH (r:Reactor)
WHERE r.Status = 'Permanent Shutdown'
RETURN count(r) AS countAfterDeletion
"""

result_after = driver.execute_query(query_count_after)
count_after = result_after[0][0]["countAfterDeletion"]

print("Number of nodes with 'Permanent Shutdown' status before deletion:", count_before)
print("Number of nodes with 'Permanent Shutdown' status after deletion:", count_after)
print("Number of nodes deleted:", count_before - count_after)


Number of nodes with 'Permanent Shutdown' status before deletion: 44
Number of nodes with 'Permanent Shutdown' status after deletion: 0
Number of nodes deleted: 44


### 4. Retrieve all reactors which located in China and have a Reactor Type of CGN.

In [55]:
query  = """
MATCH (r:Reactor {Country: 'China', `Reactor Type`: 'CGN'})
RETURN r
"""

result = driver.execute_query(query)
count_reactors_in_china_with_cgn = len(result)

print("Number of reactors in China with 'CGN' type:", count_reactors_in_china_with_cgn)

Number of reactors in China with 'CGN' type: 3


### 5. Retrieve all reactors that construction started from September 9, 2011, and May 1, 1975.

In [62]:
query = """
MATCH (r:Reactor)
WHERE r.`ConstructionStart` IN ['5/1/1975', '9/30/2011']
RETURN r
"""

result = driver.execute_query(query)
print(result)


EagerResult(records=[<Record r=<Node element_id='4:d85c10f1-8b58-4b7e-95ca-7d48097342e7:45' labels=frozenset({'Reactor'}) properties={'Status': 'Working', 'NetCapacityMWe': 660.0, 'ConstructionStart': '5/1/1975', 'ReactorType': 'PHWR', 'ReactorName': 'Point Lepreau', 'Model': 'CANDU 6', 'FirstGridConnection': '9/11/1982', 'Location': ''}>>, <Record r=<Node element_id='4:d85c10f1-8b58-4b7e-95ca-7d48097342e7:221' labels=frozenset({'Reactor'}) properties={'Status': 'Under Construction', 'NetCapacityMWe': 700.0, 'ConstructionStart': '9/30/2011', 'ReactorType': 'PHWR', 'ReactorName': 'Rajasthan 8', 'Model': 'Horizontal Pressure Tube type', 'FirstGridConnection': '', 'Location': ''}>>], summary=<neo4j._work.summary.ResultSummary object at 0x132c10f50>, keys=['r'])


### 6. How many reactors were started before 1972-08 and after the year 1975-05?

In [63]:
query_before_1972 = """
MATCH (r:Reactor)
WHERE r.`ConstructionStart` < "1972-08"
RETURN count(r) AS ReactorsBefore1972
"""

query_after_1975 = """
MATCH (r:Reactor)
WHERE r.`ConstructionStart` > "1975-05"
RETURN count(r) AS ReactorsAfter1975
"""

result_before_1972 = driver.execute_query(query_before_1972)
result_after_1975 = driver.execute_query(query_after_1975)

reactors_before_1972 = result_before_1972[0][0]
reactors_after_1975 = result_after_1975[0][0]

# Print the counts
print("Number of reactors constructed before 1972-08:", reactors_before_1972)
print("Number of reactors constructed after 1975-05:", reactors_after_1975)



Number of reactors constructed before 1972-08: <Record ReactorsBefore1972=325>
Number of reactors constructed after 1975-05: <Record ReactorsAfter1975=219>


### 7. Which is the oldest reactor still in operation?

In [65]:
query = """
MATCH (r:Reactor)
WHERE r.Status = "Working" AND r.ConstructionStart =~ ".*[0-9]{1,2}/[0-9]{1,2}/[0-9]{4}.*"
WITH r, date(datetime({year: toInteger(split(r.ConstructionStart, "/")[2]), month: toInteger(split(r.ConstructionStart, "/")[0]), day: toInteger(split(r.ConstructionStart, "/")[1])})) AS constructionDate
ORDER BY constructionDate ASC
LIMIT 1
RETURN r.ReactorName AS OldestReactorName, constructionDate
"""
result = driver.execute_query(query)

if result:
    record = result[0]
    country = record[0]
    oldest_construction_start = record[0]
    print(f"Country: {country}, Oldest Construction Start: {oldest_construction_start}")
else:
    print("No records found.")




Country: <Record OldestReactorName='Pickering 1' constructionDate=neo4j.time.Date(1966, 6, 1)>, Oldest Construction Start: <Record OldestReactorName='Pickering 1' constructionDate=neo4j.time.Date(1966, 6, 1)>


### 8. Calculate the average net capacity (MWe) for reactors of each reactor type.

In [66]:
query = """
MATCH (r:Reactor)
WHERE r.`NetCapacityMWe` IS NOT NULL
WITH r.`ReactorType` AS reactorType, toFloat(r.`Net Capacity (MWe)`) AS netCapacity
RETURN reactorType, AVG(netCapacity) AS AvgNetCapacity
"""

result = driver.execute_query(query)

print("Average net capacity of reactors by type:", result)


Average net capacity of reactors by type: EagerResult(records=[<Record reactorType='PHWR' AvgNetCapacity=None>, <Record reactorType='PWR' AvgNetCapacity=None>, <Record reactorType='' AvgNetCapacity=None>, <Record reactorType='BWR' AvgNetCapacity=None>, <Record reactorType='FBR' AvgNetCapacity=None>, <Record reactorType='1057' AvgNetCapacity=None>, <Record reactorType='720' AvgNetCapacity=None>, <Record reactorType='32' AvgNetCapacity=None>, <Record reactorType='950' AvgNetCapacity=None>, <Record reactorType='560' AvgNetCapacity=None>, <Record reactorType='820' AvgNetCapacity=None>, <Record reactorType='1417' AvgNetCapacity=None>, <Record reactorType='GCR' AvgNetCapacity=None>, <Record reactorType='1720' AvgNetCapacity=None>, <Record reactorType='OCR' AvgNetCapacity=None>, <Record reactorType='SFR' AvgNetCapacity=None>, <Record reactorType='SGR' AvgNetCapacity=None>, <Record reactorType='HTGR' AvgNetCapacity=None>], summary=<neo4j._work.summary.ResultSummary object at 0x132eccb50>, keys

### 9. Calculate the percentage of reactors in Brazil that are currently under construction.

In [68]:
query = """
MATCH (r:Reactor)-[:LOCATED_IN]->(c:Country)
WHERE c.name = 'Brazil' AND r.Status = 'Under Construction'
WITH count(r) AS underConstructionCount
MATCH (r:Reactor)-[:LOCATED_IN]->(c:Country)
WHERE c.name = 'Brazil'
WITH underConstructionCount, count(r) AS totalReactors
RETURN (toFloat(underConstructionCount) / totalReactors) * 100 AS PercentageUnderConstruction
"""

result = driver.execute_query(query)
print(result)   

EagerResult(records=[<Record PercentageUnderConstruction=100.0>], summary=<neo4j._work.summary.ResultSummary object at 0x132e5fbd0>, keys=['PercentageUnderConstruction'])


### 10. What is the percentage of reactors with PHWR (Pressurized Heavy Water Reactor) type?

In [70]:
calculate_percentage_query = """
MATCH (r:Reactor)
WHERE r.`ReactorType` = 'PHWR'
WITH count(r) AS phwrCount
MATCH (r:Reactor)
WITH phwrCount, count(r) AS totalReactors
RETURN (toFloat(phwrCount) / totalReactors) * 100 AS PercentagePHWR
"""
result = driver.execute_query(calculate_percentage_query)

print(result)

EagerResult(records=[<Record PercentagePHWR=8.02139037433155>], summary=<neo4j._work.summary.ResultSummary object at 0x130682b10>, keys=['PercentagePHWR'])


### 11. Find the top 5 combinations of location and reactor type that have the highest average net capacity (MWe), along with the names of the reactors in these combinations.

In [71]:
query = """
MATCH (r:Reactor)
RETURN r.`Location` AS Location, r.`Reactor Type` AS ReactorType, AVG(toFloat(r.`Net Capacity (MWe)`)) AS AvgNetCapacity
ORDER BY AvgNetCapacity DESC
LIMIT 5
"""

result = driver.execute_query(query)
print(result)   

EagerResult(records=[<Record Location='Fujian' ReactorType=None AvgNetCapacity=None>, <Record Location='Guangdong' ReactorType=None AvgNetCapacity=None>, <Record Location='Guangxi' ReactorType=None AvgNetCapacity=None>, <Record Location='Hainan' ReactorType=None AvgNetCapacity=None>, <Record Location='' ReactorType=None AvgNetCapacity=None>], summary=<neo4j._work.summary.ResultSummary object at 0x132ebec50>, keys=['Location', 'ReactorType', 'AvgNetCapacity'])
